# Data preparation for classifier training

This notebook shows how to prepare the data for the classifier trainig.

    Input: 
        Text file with two TAB separated columns. The first column contains the label, the second - the sentence.
    Output: 
        .json files (with train and test split) with embeddings obtained from the different pre-trained embedding models:
            1) word-level fastText embeddings: model cc.en.300.bin
                (https://fasttext.cc/docs/en/crawl-vectors.html)
            2) sentence-level transformer embeddings: model all-mpnet-base-v2
                (https://www.sbert.net/docs/pretrained_models.html#model-overview)
            3) sentence-level transformer embeddings: model all-distilroberta-v1
                (https://www.sbert.net/docs/pretrained_models.html#model-overview)
            4) sentence-level BERT cased embeddings: model BERT-Base, Cased
                (https://github.com/google-research/bert#pre-trained-models)
            5) sentence-level BERT uncased embeddings: model BERT-Base, Uncased
                (https://github.com/google-research/bert#pre-trained-models)
                
        (.._traintest.json files contain train/test split without embedding vectors)
            
  

## 1. Filtering text examples by length and syntactical structure

*** At first we tokenize text examples. Then we parse text ignoring lines containing more than 6 tokens (thouse lines are excluded from the further processing).

In [25]:
import sys
import os
import json
import csv
import re
import subprocess

In [26]:
from lambeq import SpacyTokeniser
from lambeq import BobcatParser

In [27]:
tokeniser = SpacyTokeniser()
bobcat_parser = BobcatParser(device=0)

In [28]:
datadir = "../../data/datasets"
dsName='amazonreview_train'

In [29]:
fieldnamesinfile=False
classfield="f1"
txtfield="f2"
firstsentence=False #Try to process only the first sentence for the texts longer than 6 tokens

In [30]:
input_file = f"{datadir}/{dsName}.csv"
output_file = f"{datadir}/withtags_{dsName}.tsv"

In [31]:
with open(input_file, encoding="utf8", newline='') as csvfile, open(output_file, "w", encoding="utf8") as tsvfile:
        if fieldnamesinfile != False: 
            news_reader = csv.DictReader(csvfile, delimiter=',', quotechar='"')
        else:
            news_reader = csv.DictReader(csvfile, delimiter=',', fieldnames = [classfield, txtfield], quotechar='"')        
        processed_summaries = set()
        norm_process_params = ["perl", "../../data/data_processing/scripts/normalize-punctuation.perl","-b","-l", "en"]
        norm_process = subprocess.Popen(norm_process_params, stdin=subprocess.PIPE, stdout=subprocess.PIPE, close_fds=True)
        for row in news_reader:
            score = row[classfield]
            if score == '0':
                continue
            summary = row[txtfield].replace("\n"," ").replace("\t"," ").replace("\r"," ")
            summary = re.sub('@[^\s]+ ','',summary)
            norm_process.stdin.write(summary.encode('utf-8'))
            norm_process.stdin.write('\n'.encode('utf-8'))
            norm_process.stdin.flush()
            norm_summary = norm_process.stdout.readline().decode("utf-8").rstrip()
            tok_summary = " ".join(tokeniser.tokenise_sentences([norm_summary])[0])

            if len(tok_summary.split())>6:
                if firstsentence==True:
                    tok_summary = re.sub('^([^\.!?]+).+','\\1',tok_summary)
                    if len(tok_summary.split())>6:
                        continue
                else:
                    continue
            if tok_summary in processed_summaries:
                continue
            sent_type = ''
            processed_summaries.add(tok_summary)
            try:
                sent_type = 's'
                result = bobcat_parser.sentence2tree(tok_summary).to_json()
                str1=re.sub('\'(rule|text)\':\s[\"\'][^\s]+[\"\']','',str(result))
                str1=re.sub('\'(type|children)\':\s+','',str1)
                str1=re.sub('[\{\},\']','',str1)
                str1=re.sub('\s+([\[\]])','\\1',str1)
                sent_type = str1
            except:
                sent_type = ''
            print("{0}\t{1}\t{2}".format(score, tok_summary,sent_type),file=tsvfile)

Output is 3-column filtered file containing class, text and syntactical structure of the text.

*** Next we chose text examples according to the set of prespecified syntactical structure.

In [47]:
input_file = f"{datadir}/withtags_{dsName}.tsv"
output_file = f"{datadir}/{dsName}.tsv"
tags_file = f"{datadir}/tags.txt"

In [48]:
filterlist = open(tags_file).read().splitlines()
print(filterlist)

['n[(n/n)   n[(n/n)   n]]', 'n[(n/n)   n[(n/n)   n[(n/n)\xa0\xa0 n]]]', 'n[n[n[(n/n)   n]] (n\\\\n)[((n\\\\n)/n)   n[n[(n/n)   n]]]]', 'n[(n/n)[((n/n)/(n/n))   (n/n)] n]', 'n[n[n[(n/n)   n]] (n\\\\n)[((n\\\\n)/n)   n[(n/n)   n]]]']


In [49]:
  
with open(input_file, "r", encoding="utf8") as ifile, open(output_file, "w", encoding="utf8") as ofile:
    tsv_reader = csv.DictReader(ifile, fieldnames=['Class','Txt','Tag'], delimiter="\t", quotechar='"')
    for item in tsv_reader:
        if item['Tag'] in filterlist:
            print("{0}\t{1}".format(item['Class'],item['Txt']),file=ofile)

Output is 2-column filtered file containing class and text.

## 2. Splitting examples in train and test sets and acquiring embedding vectors

In [50]:
from sklearn.model_selection import train_test_split
from typing import List, Tuple, Dict
sys.path.append("../../data/data_processing/data_vectorisation/")
from Embeddings import Embeddings
from collections import defaultdict

In [51]:
def unpack_data(data: List[Tuple[str, str]]) -> List[Dict[str, str]]:
    return [{
        "sentence": sentence,
        "class": sentence_type,
    } for sentence, sentence_type in data]

In [52]:
input_file = f"{datadir}/{dsName}.tsv"

In [53]:
dataset: List[Tuple[str, str]] = []

In [54]:
with open(input_file, "r", encoding="utf-8") as f:
    for line in f:
        cols=line.split('\t')
        if len(cols) == 2:
            sent = cols[1].rstrip()
            dataset.append((sent, cols[0].rstrip()))


In [55]:
classes = [item[1] for item in dataset]
classes

['2',
 '2',
 '2',
 '1',
 '2',
 '1',
 '2',
 '1',
 '2',
 '1',
 '1',
 '1',
 '2',
 '1',
 '2',
 '2',
 '1',
 '2',
 '1',
 '2',
 '1',
 '2',
 '2',
 '2',
 '1',
 '1',
 '2',
 '2',
 '2',
 '2',
 '2',
 '1',
 '1',
 '1',
 '2',
 '1',
 '2',
 '2',
 '2',
 '1',
 '1',
 '2',
 '2',
 '2',
 '1',
 '1',
 '1',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '1',
 '2',
 '1',
 '1',
 '1',
 '1',
 '2',
 '1',
 '1',
 '2',
 '2',
 '2',
 '2',
 '1',
 '1',
 '1',
 '1',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '1',
 '1',
 '1',
 '1',
 '2',
 '1',
 '2',
 '2',
 '1',
 '1',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '1',
 '2',
 '2',
 '2',
 '2',
 '2',
 '1',
 '2',
 '2',
 '2',
 '2',
 '2',
 '1',
 '1',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '1',
 '1',
 '1',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '1',
 '1',
 '2',
 '2',
 '1',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '1',
 '1',
 '2',
 '1',
 '2',
 '2',
 '2',
 '2',
 '2',
 '1',
 '1',
 '2',
 '1',
 '2',
 '1',
 '2',
 '1',
 '1'

In [56]:
train_data, test_data = train_test_split(dataset, train_size=0.9, random_state=1, stratify=classes)

In [57]:
my_result = defaultdict(list)
for element in test_data:
    my_result[element[1]].append(element[0])

my_result = dict(my_result)
result_dictionary = dict()

for key in my_result:
    result_dictionary[key] = len(list(set(my_result[key]))) / len(test_data)
print(f"*** Proportion of classes in {len(test_data)} examples of test data ***")
print(json.dumps(result_dictionary, indent=4, sort_keys=True))

*** Proportion of classes in 16179 examples of test data ***
{
    "1": 0.3466221645342728,
    "2": 0.6533778354657271
}


In [58]:
my_result = defaultdict(list)
for element in train_data:
    my_result[element[1]].append(element[0])

my_result = dict(my_result)
result_dictionary = dict()

for key in my_result:
    result_dictionary[key] = len(list(set(my_result[key]))) / len(train_data)
print(f"*** Proportion of classes in {len(train_data)} examples of train data ***")
print(json.dumps(result_dictionary, indent=4, sort_keys=True))

*** Proportion of classes in 145610 examples of train data ***
{
    "1": 0.34662454501751255,
    "2": 0.6533754549824875
}


In [59]:
test_data = unpack_data(test_data)

In [60]:
train_data = unpack_data(train_data)

In [61]:
with open(f"{datadir}/{dsName}_traintest.json", "w", encoding="utf-8") as f:
        json.dump({"train_data": train_data, "test_data": test_data}, f, indent=1, ensure_ascii=False)

In [62]:
def ObtainEmbeddings(train_data, test_data, key, path, embtype):
    vectorizer = Embeddings(path=path,embtype=embtype)
        
    cnt = 0
    print(f"\n*** Getting vectors for {len(train_data)} examples of train data ***", end='\n')
    for item in train_data:
        item["sentence_vectorized"] = vectorizer.getEmbeddingVector(item["sentence"])
        cnt = cnt + 1
        if cnt % 50 == 0:
            print (str(cnt),end=' ')
                
    cnt = 0
    print(f"\n*** Getting vectors for {len(test_data)} examples of test data ***", end='\n')
    for item in test_data:
        item["sentence_vectorized"] = vectorizer.getEmbeddingVector(item["sentence"])
        cnt = cnt + 1
        if cnt % 50 == 0:
            print (str(cnt),end=' ')
        
    with open(f"{datadir}/{dsName}_{key}.json", "w", encoding="utf-8") as f:
        json.dump({"train_data": train_data, "test_data": test_data}, f, indent=2, ensure_ascii=False)

In [63]:
ObtainEmbeddings(train_data, test_data, 'FASTTEXT', 'cc.en.300.bin', 'fasttext')

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

 (100.00%) [==================================================>]================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=================================================> ]=

cc.en.300.bin loaded!

*** Getting vectors for 145610 examples of train data ***


50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 1000 1050 1100 1150 1200 1250 1300 1350 1400 1450 1500 1550 1600 1650 1700 1750 1800 1850 1900 1950 2000 2050 2100 2150 2200 2250 2300 2350 2400 2450 2500 2550 2600 2650 2700 2750 2800 2850 2900 2950 3000 3050 3100 3150 3200 3250 3300 3350 3400 3450 3500 3550 3600 3650 3700 3750 3800 3850 3900 3950 4000 4050 4100 4150 4200 4250 4300 4350 4400 4450 4500 4550 4600 4650 4700 4750 4800 4850 4900 4950 5000 5050 5100 5150 5200 5250 5300 5350 5400 5450 5500 5550 5600 5650 5700 5750 5800 5850 5900 5950 6000 6050 6100 6150 6200 6250 6300 6350 6400 6450 6500 6550 6600 6650 6700 6750 6800 6850 6900 6950 7000 7050 7100 7150 7200 7250 7300 7350 7400 7450 7500 7550 7600 7650 7700 7750 7800 7850 7900 7950 8000 8050 8100 8150 8200 8250 8300 8350 8400 8450 8500 8550 8600 8650 8700 8750 8800 8850 8900 8950 9000 9050 9100 9150 9200 9250 9300 9350 9400 9450 9500 9550 9600 9650 9700 9750 9800 9850 9900 9950 10000 10050 10100 10150 1

84300 84350 84400 84450 84500 84550 84600 84650 84700 84750 84800 84850 84900 84950 85000 85050 85100 85150 85200 85250 85300 85350 85400 85450 85500 85550 85600 85650 85700 85750 85800 85850 85900 85950 86000 86050 86100 86150 86200 86250 86300 86350 86400 86450 86500 86550 86600 86650 86700 86750 86800 86850 86900 86950 87000 87050 87100 87150 87200 87250 87300 87350 87400 87450 87500 87550 87600 87650 87700 87750 87800 87850 87900 87950 88000 88050 88100 88150 88200 88250 88300 88350 88400 88450 88500 88550 88600 88650 88700 88750 88800 88850 88900 88950 89000 89050 89100 89150 89200 89250 89300 89350 89400 89450 89500 89550 89600 89650 89700 89750 89800 89850 89900 89950 90000 90050 90100 90150 90200 90250 90300 90350 90400 90450 90500 90550 90600 90650 90700 90750 90800 90850 90900 90950 91000 91050 91100 91150 91200 91250 91300 91350 91400 91450 91500 91550 91600 91650 91700 91750 91800 91850 91900 91950 92000 92050 92100 92150 92200 92250 92300 92350 92400 92450 92500 92550 9260

50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 1000 1050 1100 1150 1200 1250 1300 1350 1400 1450 1500 1550 1600 1650 1700 1750 1800 1850 1900 1950 2000 2050 2100 2150 2200 2250 2300 2350 2400 2450 2500 2550 2600 2650 2700 2750 2800 2850 2900 2950 3000 3050 3100 3150 3200 3250 3300 3350 3400 3450 3500 3550 3600 3650 3700 3750 3800 3850 3900 3950 4000 4050 4100 4150 4200 4250 4300 4350 4400 4450 4500 4550 4600 4650 4700 4750 4800 4850 4900 4950 5000 5050 5100 5150 5200 5250 5300 5350 5400 5450 5500 5550 5600 5650 5700 5750 5800 5850 5900 5950 6000 6050 6100 6150 6200 6250 6300 6350 6400 6450 6500 6550 6600 6650 6700 6750 6800 6850 6900 6950 7000 7050 7100 7150 7200 7250 7300 7350 7400 7450 7500 7550 7600 7650 7700 7750 7800 7850 7900 7950 8000 8050 8100 8150 8200 8250 8300 8350 8400 8450 8500 8550 8600 8650 8700 8750 8800 8850 8900 8950 9000 9050 9100 9150 9200 9250 9300 9350 9400 9450 9500 9550 9600 9650 9700 9750 9800 9850 9900 9950 10000 10050 10100 10150 1

In [64]:
ObtainEmbeddings(train_data, test_data, 'all-mpnet-base', 'all-mpnet-base-v2', 'transformer')

all-mpnet-base-v2 loaded!

*** Getting vectors for 145610 examples of train data ***


50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 1000 1050 1100 1150 1200 1250 1300 1350 1400 1450 1500 1550 1600 1650 1700 1750 1800 1850 1900 1950 2000 2050 2100 2150 2200 2250 2300 2350 2400 2450 2500 2550 2600 2650 2700 2750 2800 2850 2900 2950 3000 3050 3100 3150 3200 3250 3300 3350 3400 3450 3500 3550 3600 3650 3700 3750 3800 3850 3900 3950 4000 4050 4100 4150 4200 4250 4300 4350 4400 4450 4500 4550 4600 4650 4700 4750 4800 4850 4900 4950 5000 5050 5100 5150 5200 5250 5300 5350 5400 5450 5500 5550 5600 5650 5700 5750 5800 5850 5900 5950 6000 6050 6100 6150 6200 6250 6300 6350 6400 6450 6500 6550 6600 6650 6700 6750 6800 6850 6900 6950 7000 7050 7100 7150 7200 7250 7300 7350 7400 7450 7500 7550 7600 7650 7700 7750 7800 7850 7900 7950 8000 8050 8100 8150 8200 8250 8300 8350 8400 8450 8500 8550 8600 8650 8700 8750 8800 8850 8900 8950 9000 9050 9100 9150 9200 9250 9300 9350 9400 9450 9500 9550 9600 9650 9700 9750 9800 9850 9900 9950 10000 10050 10100 10150 1

70150 70200 70250 70300 70350 70400 70450 70500 70550 70600 70650 70700 70750 70800 70850 70900 70950 71000 71050 71100 71150 71200 71250 71300 71350 71400 71450 71500 71550 71600 71650 71700 71750 71800 71850 71900 71950 72000 72050 72100 72150 72200 72250 72300 72350 72400 72450 72500 72550 72600 72650 72700 72750 72800 72850 72900 72950 73000 73050 73100 73150 73200 73250 73300 73350 73400 73450 73500 73550 73600 73650 73700 73750 73800 73850 73900 73950 74000 74050 74100 74150 74200 74250 74300 74350 74400 74450 74500 74550 74600 74650 74700 74750 74800 74850 74900 74950 75000 75050 75100 75150 75200 75250 75300 75350 75400 75450 75500 75550 75600 75650 75700 75750 75800 75850 75900 75950 76000 76050 76100 76150 76200 76250 76300 76350 76400 76450 76500 76550 76600 76650 76700 76750 76800 76850 76900 76950 77000 77050 77100 77150 77200 77250 77300 77350 77400 77450 77500 77550 77600 77650 77700 77750 77800 77850 77900 77950 78000 78050 78100 78150 78200 78250 78300 78350 78400 7845

132950 133000 133050 133100 133150 133200 133250 133300 133350 133400 133450 133500 133550 133600 133650 133700 133750 133800 133850 133900 133950 134000 134050 134100 134150 134200 134250 134300 134350 134400 134450 134500 134550 134600 134650 134700 134750 134800 134850 134900 134950 135000 135050 135100 135150 135200 135250 135300 135350 135400 135450 135500 135550 135600 135650 135700 135750 135800 135850 135900 135950 136000 136050 136100 136150 136200 136250 136300 136350 136400 136450 136500 136550 136600 136650 136700 136750 136800 136850 136900 136950 137000 137050 137100 137150 137200 137250 137300 137350 137400 137450 137500 137550 137600 137650 137700 137750 137800 137850 137900 137950 138000 138050 138100 138150 138200 138250 138300 138350 138400 138450 138500 138550 138600 138650 138700 138750 138800 138850 138900 138950 139000 139050 139100 139150 139200 139250 139300 139350 139400 139450 139500 139550 139600 139650 139700 139750 139800 139850 139900 139950 140000 140050

In [65]:
ObtainEmbeddings(train_data, test_data, 'all-distilroberta', 'all-distilroberta-v1', 'transformer')

all-distilroberta-v1 loaded!

*** Getting vectors for 145610 examples of train data ***


50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 1000 1050 1100 1150 1200 1250 1300 1350 1400 1450 1500 1550 1600 1650 1700 1750 1800 1850 1900 1950 2000 2050 2100 2150 2200 2250 2300 2350 2400 2450 2500 2550 2600 2650 2700 2750 2800 2850 2900 2950 3000 3050 3100 3150 3200 3250 3300 3350 3400 3450 3500 3550 3600 3650 3700 3750 3800 3850 3900 3950 4000 4050 4100 4150 4200 4250 4300 4350 4400 4450 4500 4550 4600 4650 4700 4750 4800 4850 4900 4950 5000 5050 5100 5150 5200 5250 5300 5350 5400 5450 5500 5550 5600 5650 5700 5750 5800 5850 5900 5950 6000 6050 6100 6150 6200 6250 6300 6350 6400 6450 6500 6550 6600 6650 6700 6750 6800 6850 6900 6950 7000 7050 7100 7150 7200 7250 7300 7350 7400 7450 7500 7550 7600 7650 7700 7750 7800 7850 7900 7950 8000 8050 8100 8150 8200 8250 8300 8350 8400 8450 8500 8550 8600 8650 8700 8750 8800 8850 8900 8950 9000 9050 9100 9150 9200 9250 9300 9350 9400 9450 9500 9550 9600 9650 9700 9750 9800 9850 9900 9950 10000 10050 10100 10150 1

70150 70200 70250 70300 70350 70400 70450 70500 70550 70600 70650 70700 70750 70800 70850 70900 70950 71000 71050 71100 71150 71200 71250 71300 71350 71400 71450 71500 71550 71600 71650 71700 71750 71800 71850 71900 71950 72000 72050 72100 72150 72200 72250 72300 72350 72400 72450 72500 72550 72600 72650 72700 72750 72800 72850 72900 72950 73000 73050 73100 73150 73200 73250 73300 73350 73400 73450 73500 73550 73600 73650 73700 73750 73800 73850 73900 73950 74000 74050 74100 74150 74200 74250 74300 74350 74400 74450 74500 74550 74600 74650 74700 74750 74800 74850 74900 74950 75000 75050 75100 75150 75200 75250 75300 75350 75400 75450 75500 75550 75600 75650 75700 75750 75800 75850 75900 75950 76000 76050 76100 76150 76200 76250 76300 76350 76400 76450 76500 76550 76600 76650 76700 76750 76800 76850 76900 76950 77000 77050 77100 77150 77200 77250 77300 77350 77400 77450 77500 77550 77600 77650 77700 77750 77800 77850 77900 77950 78000 78050 78100 78150 78200 78250 78300 78350 78400 7845

132950 133000 133050 133100 133150 133200 133250 133300 133350 133400 133450 133500 133550 133600 133650 133700 133750 133800 133850 133900 133950 134000 134050 134100 134150 134200 134250 134300 134350 134400 134450 134500 134550 134600 134650 134700 134750 134800 134850 134900 134950 135000 135050 135100 135150 135200 135250 135300 135350 135400 135450 135500 135550 135600 135650 135700 135750 135800 135850 135900 135950 136000 136050 136100 136150 136200 136250 136300 136350 136400 136450 136500 136550 136600 136650 136700 136750 136800 136850 136900 136950 137000 137050 137100 137150 137200 137250 137300 137350 137400 137450 137500 137550 137600 137650 137700 137750 137800 137850 137900 137950 138000 138050 138100 138150 138200 138250 138300 138350 138400 138450 138500 138550 138600 138650 138700 138750 138800 138850 138900 138950 139000 139050 139100 139150 139200 139250 139300 139350 139400 139450 139500 139550 139600 139650 139700 139750 139800 139850 139900 139950 140000 140050

In [66]:
ObtainEmbeddings(train_data, test_data, 'BERT_UNCASED', 'bert-base-uncased', 'bert')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


bert-base-uncased loaded!

*** Getting vectors for 145610 examples of train data ***


50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 1000 1050 1100 1150 1200 1250 1300 1350 1400 1450 1500 1550 1600 1650 1700 1750 1800 1850 1900 1950 2000 2050 2100 2150 2200 2250 2300 2350 2400 2450 2500 2550 2600 2650 2700 2750 2800 2850 2900 2950 3000 3050 3100 3150 3200 3250 3300 3350 3400 3450 3500 3550 3600 3650 3700 3750 3800 3850 3900 3950 4000 4050 4100 4150 4200 4250 4300 4350 4400 4450 4500 4550 4600 4650 4700 4750 4800 4850 4900 4950 5000 5050 5100 5150 5200 5250 5300 5350 5400 5450 5500 5550 5600 5650 5700 5750 5800 5850 5900 5950 6000 6050 6100 6150 6200 6250 6300 6350 6400 6450 6500 6550 6600 6650 6700 6750 6800 6850 6900 6950 7000 7050 7100 7150 7200 7250 7300 7350 7400 7450 7500 7550 7600 7650 7700 7750 7800 7850 7900 7950 8000 8050 8100 8150 8200 8250 8300 8350 8400 8450 8500 8550 8600 8650 8700 8750 8800 8850 8900 8950 9000 9050 9100 9150 9200 9250 9300 9350 9400 9450 9500 9550 9600 9650 9700 9750 9800 9850 9900 9950 10000 10050 10100 10150 1

70150 70200 70250 70300 70350 70400 70450 70500 70550 70600 70650 70700 70750 70800 70850 70900 70950 71000 71050 71100 71150 71200 71250 71300 71350 71400 71450 71500 71550 71600 71650 71700 71750 71800 71850 71900 71950 72000 72050 72100 72150 72200 72250 72300 72350 72400 72450 72500 72550 72600 72650 72700 72750 72800 72850 72900 72950 73000 73050 73100 73150 73200 73250 73300 73350 73400 73450 73500 73550 73600 73650 73700 73750 73800 73850 73900 73950 74000 74050 74100 74150 74200 74250 74300 74350 74400 74450 74500 74550 74600 74650 74700 74750 74800 74850 74900 74950 75000 75050 75100 75150 75200 75250 75300 75350 75400 75450 75500 75550 75600 75650 75700 75750 75800 75850 75900 75950 76000 76050 76100 76150 76200 76250 76300 76350 76400 76450 76500 76550 76600 76650 76700 76750 76800 76850 76900 76950 77000 77050 77100 77150 77200 77250 77300 77350 77400 77450 77500 77550 77600 77650 77700 77750 77800 77850 77900 77950 78000 78050 78100 78150 78200 78250 78300 78350 78400 7845

132950 133000 133050 133100 133150 133200 133250 133300 133350 133400 133450 133500 133550 133600 133650 133700 133750 133800 133850 133900 133950 134000 134050 134100 134150 134200 134250 134300 134350 134400 134450 134500 134550 134600 134650 134700 134750 134800 134850 134900 134950 135000 135050 135100 135150 135200 135250 135300 135350 135400 135450 135500 135550 135600 135650 135700 135750 135800 135850 135900 135950 136000 136050 136100 136150 136200 136250 136300 136350 136400 136450 136500 136550 136600 136650 136700 136750 136800 136850 136900 136950 137000 137050 137100 137150 137200 137250 137300 137350 137400 137450 137500 137550 137600 137650 137700 137750 137800 137850 137900 137950 138000 138050 138100 138150 138200 138250 138300 138350 138400 138450 138500 138550 138600 138650 138700 138750 138800 138850 138900 138950 139000 139050 139100 139150 139200 139250 139300 139350 139400 139450 139500 139550 139600 139650 139700 139750 139800 139850 139900 139950 140000 140050

In [67]:
ObtainEmbeddings(train_data, test_data, 'BERT_CASED', 'bert-base-cased', 'bert')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


bert-base-cased loaded!

*** Getting vectors for 145610 examples of train data ***


50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 800 850 900 950 1000 1050 1100 1150 1200 1250 1300 1350 1400 1450 1500 1550 1600 1650 1700 1750 1800 1850 1900 1950 2000 2050 2100 2150 2200 2250 2300 2350 2400 2450 2500 2550 2600 2650 2700 2750 2800 2850 2900 2950 3000 3050 3100 3150 3200 3250 3300 3350 3400 3450 3500 3550 3600 3650 3700 3750 3800 3850 3900 3950 4000 4050 4100 4150 4200 4250 4300 4350 4400 4450 4500 4550 4600 4650 4700 4750 4800 4850 4900 4950 5000 5050 5100 5150 5200 5250 5300 5350 5400 5450 5500 5550 5600 5650 5700 5750 5800 5850 5900 5950 6000 6050 6100 6150 6200 6250 6300 6350 6400 6450 6500 6550 6600 6650 6700 6750 6800 6850 6900 6950 7000 7050 7100 7150 7200 7250 7300 7350 7400 7450 7500 7550 7600 7650 7700 7750 7800 7850 7900 7950 8000 8050 8100 8150 8200 8250 8300 8350 8400 8450 8500 8550 8600 8650 8700 8750 8800 8850 8900 8950 9000 9050 9100 9150 9200 9250 9300 9350 9400 9450 9500 9550 9600 9650 9700 9750 9800 9850 9900 9950 10000 10050 10100 10150 1

70150 70200 70250 70300 70350 70400 70450 70500 70550 70600 70650 70700 70750 70800 70850 70900 70950 71000 71050 71100 71150 71200 71250 71300 71350 71400 71450 71500 71550 71600 71650 71700 71750 71800 71850 71900 71950 72000 72050 72100 72150 72200 72250 72300 72350 72400 72450 72500 72550 72600 72650 72700 72750 72800 72850 72900 72950 73000 73050 73100 73150 73200 73250 73300 73350 73400 73450 73500 73550 73600 73650 73700 73750 73800 73850 73900 73950 74000 74050 74100 74150 74200 74250 74300 74350 74400 74450 74500 74550 74600 74650 74700 74750 74800 74850 74900 74950 75000 75050 75100 75150 75200 75250 75300 75350 75400 75450 75500 75550 75600 75650 75700 75750 75800 75850 75900 75950 76000 76050 76100 76150 76200 76250 76300 76350 76400 76450 76500 76550 76600 76650 76700 76750 76800 76850 76900 76950 77000 77050 77100 77150 77200 77250 77300 77350 77400 77450 77500 77550 77600 77650 77700 77750 77800 77850 77900 77950 78000 78050 78100 78150 78200 78250 78300 78350 78400 7845

132950 133000 133050 133100 133150 133200 133250 133300 133350 133400 133450 133500 133550 133600 133650 133700 133750 133800 133850 133900 133950 134000 134050 134100 134150 134200 134250 134300 134350 134400 134450 134500 134550 134600 134650 134700 134750 134800 134850 134900 134950 135000 135050 135100 135150 135200 135250 135300 135350 135400 135450 135500 135550 135600 135650 135700 135750 135800 135850 135900 135950 136000 136050 136100 136150 136200 136250 136300 136350 136400 136450 136500 136550 136600 136650 136700 136750 136800 136850 136900 136950 137000 137050 137100 137150 137200 137250 137300 137350 137400 137450 137500 137550 137600 137650 137700 137750 137800 137850 137900 137950 138000 138050 138100 138150 138200 138250 138300 138350 138400 138450 138500 138550 138600 138650 138700 138750 138800 138850 138900 138950 139000 139050 139100 139150 139200 139250 139300 139350 139400 139450 139500 139550 139600 139650 139700 139750 139800 139850 139900 139950 140000 140050